In [1]:
# For any HF basic activities like loading models
# and tokenizers for running inference
# upgrade is a must for the newest Gemma model
!pip install --upgrade datasets
!pip install --upgrade transformers

# For doing efficient stuff - PEFT
!pip install --upgrade peft
!pip install --upgrade trl
!pip install bitsandbytes
!pip install accelerate

# for logging and visualizing training progress
!pip install tensorboard
# If creating a new dataset, useful for creating *.jsonl files
!pip install jsonlines

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_syst

In [2]:
from huggingface_hub import notebook_login
from google.colab import userdata
hf_token = userdata.get('hftoken')
notebook_login()

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# model_name = "google/gemma-2b"
model_name = "meta-llama/Llama-3.2-1B"
# model_name = "meta-llama/Llama-2-7b-hf"

model = AutoModelForCausalLM.from_pretrained(model_name,
                                            #  torch_dtype=torch.float16
                                             )
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          # torch_dtype=torch.float16
                                          )
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rotary_emb):

In [5]:
input_text = "I want to eat Pizza"

input_ids = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(**input_ids, max_length=128)
print(tokenizer.decode(outputs[0]))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>I want to eat Pizza Hut’s Pizza for dinner tonight. I want to eat Pizza Hut’s Pizza for dinner tonight. I want to eat Pizza Hut’s Pizza for dinner tonight. I want to eat Pizza Hut’s Pizza for dinner tonight. I want to eat Pizza Hut’s Pizza for dinner tonight. I want to eat Pizza Hut’s Pizza for dinner tonight. I want to eat Pizza Hut’s Pizza for dinner tonight. I want to eat Pizza Hut’s Pizza for dinner tonight. I want to eat Pizza Hut’s Pizza for dinner tonight. I want to eat Pizza Hut’s Pizza for dinner tonight. I want to eat Pizza Hut’s


In [8]:
input_text = "I want to eat pasta. My budget is 200Rupees for 1 person."
# input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")
input_ids = tokenizer(input_text, return_tensors="pt")
print(input_ids)
outputs = model.generate(**input_ids, max_length=128)
print(tokenizer.decode(outputs[0]))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'input_ids': tensor([[128000,     40,   1390,    311,   8343,  37547,     13,   3092,   8199,
            374,    220,   1049,  99725,  82400,    369,    220,     16,   1732,
             13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
<|begin_of_text|>I want to eat pasta. My budget is 200Rupees for 1 person. I want to eat pasta 3 times a week. Can you please give me a list of the cheapest places to eat pasta in Mumbai?
I want to eat pasta. My budget is 200Rupees for 1 person. I want to eat pasta 3 times a week. Can you please give me a list of the cheapest places to eat pasta in Mumbai?
You can try these 3 places:
1.   Pasta & Co., 1st floor, Kurla, Thane West, Mumbai. 400 070


In [16]:
from datasets import load_dataset

dataset_name = "/content/food_agent_data.csv"
# dataset = load_dataset(dataset_name, split="train[0:1000]")
dataset = load_dataset("csv", data_files=dataset_name, split="train[0:1000]")

print(f"Instruction is: {dataset[0]['prompt']}")
print(f"Response is: {dataset[0]['response']}")
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Instruction is: What kind of dish are you in the mood for?
Response is: What type of dish would you like? (e.g., Italian, Mexican, Indian)


Dataset({
    features: ['prompt', 'response'],
    num_rows: 63
})

In [20]:
def formatting_prompts_func(example):
  output_texts = []
  for i in range(len(example['prompt'])):
    text = f"Instruction:\n{example['prompt']}\n\nResponse:\n{example['response']}"
    output_texts.append(text)
  return output_texts

from trl import SFTTrainer
trainer = SFTTrainer(
    model,
    train_dataset=dataset,
    tokenizer=tokenizer,
    formatting_func=formatting_prompts_func,
)
print("Initialized trainer for training!")

trainer.train()

<ipython-input-20-067c6c599e81>:9: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Map:   0%|          | 0/63 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Initialized trainer for training!


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: varuntej2480 (varuntej2480-ltimindtree) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.

In [21]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [23]:
from datasets import load_dataset

dataset_name = "/content/food_agent_data.csv"
dataset = load_dataset("csv", data_files=dataset_name, split="train[0:1000]")

In [25]:

filtered_dataset = []
for __, data in enumerate(dataset):
  text = f"Instruction:\n{data['prompt']}\n\nResponse:\n{data['response']}"
  filtered_dataset.append({"text": text})

print(filtered_dataset[0:2])

[{'text': 'Instruction:\nWhat kind of dish are you in the mood for?\n\nResponse:\nWhat type of dish would you like? (e.g., Italian, Mexican, Indian)'}, {'text': "Instruction:\nI'd like something Italian.\n\nResponse:\nGreat! And what kind of Italian dish are you thinking of? (e.g., pasta, pizza, risotto)"}]


In [26]:
# convert to json and save the filtered dataset as jsonl file
import jsonlines as jl
with jl.open('food_agent_data.jsonl', 'w') as writer:
    writer.write_all(filtered_dataset[0:])

In [40]:
from datasets import load_dataset

dataset_name = "/content/food_agent_data.jsonl"
# dataset = load_dataset(dataset_name, split="train")
dataset = load_dataset('json', data_files=dataset_name, split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [63]:
# define some variables - model names
# model_name = "google/gemma-2b"
model_name = "meta-llama/Llama-3.2-1B"
new_model = "Marco-by-llama-3.2-1B"

################################################################################
# LoRA parameters
################################################################################
# LoRA attention dimension
# lora_r = 64
lora_r = 4
# Alpha parameter for LoRA scaling
lora_alpha = 16
# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################
# Activate 4-bit precision base model loading
use_4bit = True
# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"
# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"
# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################
# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"
# Number of training epochs
num_train_epochs = 1
# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False
# Batch size per GPU for training
per_device_train_batch_size = 4
# Batch size per GPU for evaluation
per_device_eval_batch_size = 4
# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1
# Enable gradient checkpointing
gradient_checkpointing = True
# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3
# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4
# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001
# Optimizer to use
optim = "paged_adamw_32bit"
# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"
# Number of training steps (overrides num_train_epochs)
max_steps = -1
# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03
# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True
# Save checkpoint every X updates steps
save_steps = 25
# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################
# Maximum sequence length to use
max_seq_length = 40 # None
# Pack multiple short examples in the same input sequence to increase efficiency
packing = True # False
# Load the entire model on the GPU 0
# device_map = {"": 0}
device_map="auto"

In [64]:
# Load QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit, # Activates 4-bit precision loading
    bnb_4bit_quant_type=bnb_4bit_quant_type, # nf4
    bnb_4bit_compute_dtype=compute_dtype, # float16
    bnb_4bit_use_double_quant=use_nested_quant, # False
)

In [65]:
# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("Setting BF16 to True")
        bf16 = True
    else:
        bf16 = False

In [67]:
# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    token=hf_token,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          token=hf_token,
                                          trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

In [68]:
# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj", "up_proj"]
)

In [69]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",
)
training_arguments

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=IntervalStrategy.NO,
eval_us

In [70]:
from trl import SFTConfig

config = SFTConfig(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    packing=packing,

)

In [71]:
# Set supervised fine-tuning parameters


trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    # dataset_text_field="text",
    # formatting_func=format_prompts_fn,
    # max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    # args=training_arguments,
    args=config,
    # packing=packing,
)

<ipython-input-71-38f0e2722d1d>:4: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


In [72]:
# Train model
trainer.train()
trainer.model.save_pretrained(new_model)

Step,Training Loss


In [73]:

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [74]:
input_text = "I want to eat Pizza."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")
print(input_ids)
outputs = model.generate(**input_ids, max_length=128)
print(tokenizer.decode(outputs[0]))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'input_ids': tensor([[128000,     40,   1390,    311,   8343,  35918,     13]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}
<|begin_of_text|>I want to eat Pizza.<|end_of_text|>


In [75]:
input_text = "I want to eat pasta. My budget is 200Rupees for 1 person."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")
# input_ids = tokenizer(input_text, return_tensors="pt")
print(input_ids)
outputs = model.generate(**input_ids, max_length=128)
print(tokenizer.decode(outputs[0]))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'input_ids': tensor([[128000,     40,   1390,    311,   8343,  37547,     13,   3092,   8199,
            374,    220,   1049,  99725,  82400,    369,    220,     16,   1732,
             13]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}
<|begin_of_text|>I want to eat pasta. My budget is 200Rupees for 1 person.<|end_of_text|>


#Llama 3.1 Fine Tune

In [77]:
pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 3.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 kB 12.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.6
    Uninstalling protobuf-4.25.6:
      Successfully uninstalled protobuf-4.25.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of

In [78]:
from unsloth import FastLanguageModel
import torch
import os
from transformers import TextStreamer
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [82]:
# 1. Configuration
max_seq_length = 2048
dtype = None
load_in_4bit = True
marco_prompt = """Below is an instruction that describes a user query, Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""
instruction = "I want to eat pizza."
# huggingface_model_name = "mervinpraison/Llama-3.1-8B-bnb-4bit-python"

In [83]:
# 2. Before Training
model, tokenizer = FastLanguageModel.from_pretrained(
    # model_name = "unsloth/Llama-3.2-1B-Instruct-GGUF",
    model_name = "unsloth/Llama-3.2-1B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = os.getenv("HF_TOKEN")
)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    marco_prompt.format(
        instruction, # instruction
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 1000)

==((====))==  Unsloth 2025.1.8: Fast Llama patching. Transformers: 4.48.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
<|begin_of_text|>Below is an instruction that describes a user query, Write a response that appropriately completes the request.

### Instruction:
I want to eat pizza.

### Response:
I'm happy to help you with your request. Here's a suggested response that completes the request:

"Would you like to order a large pizza with toppings of your choice, or would you prefer to pick up one from a nearby pizzeria?"<|eot_id|>


In [84]:
# 3. Load data

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
# def formatting_prompts_func(examples):
#     instructions = examples["instruction"]
#     inputs       = examples["input"]
#     outputs      = examples["output"]
#     texts = []
#     for instruction, input, output in zip(instructions, inputs, outputs):
#         # text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
#         text = f"Instruction:\n{data['prompt']}\n\nResponse:\n{data['response']}" + EOS_TOKEN
#         texts.append(text)
#     return { "text" : texts, }

def formatting_prompts_func(example):
  output_texts = []
  for i in range(len(example['prompt'])):
    text = f"Instruction:\n{example['prompt']}\n\nResponse:\n{example['response']}" + EOS_TOKEN
    output_texts.append(text)
  return { "text" : output_texts, }

# dataset = load_dataset("iamtarun/python_code_instructions_18k_alpaca", split = "train")
dataset_name = "/content/food_agent_data.csv"
dataset = load_dataset("csv", data_files=dataset_name, split="train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/63 [00:00<?, ? examples/s]

In [85]:
# 4. Training
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)


Unsloth 2025.1.8 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


In [86]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 100,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/63 [00:00<?, ? examples/s]

In [87]:
# Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

trainer_stats = trainer.train()

GPU = Tesla T4. Max memory = 14.741 GB.
7.949 GB of memory reserved.


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 63 | Num Epochs = 13
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 100
 "-____-"     Number of trainable parameters = 11,272,192


Step,Training Loss
1,1.157100
2,1.157100
3,1.144900
4,1.102500
5,1.040200
6,0.961700
7,0.868600
8,0.780500
9,0.695500
10,0.612500


In [88]:
# Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

857.6469 seconds used for training.
14.29 minutes used for training.
Peak reserved memory = 7.949 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 53.924 %.
Peak reserved memory for training % of max memory = 0.0 %.


In [89]:
# 5. After Training
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    marco_prompt.format(
        instruction, # instruction
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 1000)

<|begin_of_text|>Below is an instruction that describes a user query, Write a response that appropriately completes the request.

### Instruction:
I want to eat pizza.

### Response:
"Okay, what kind of pizza are you in the mood for? Italian, Mexican, or something else?" 

This response acknowledges the user's request for pizza, and opens it up to a variety of pizza options. It also shows that you're willing to consider different types of pizza, which may help the user narrow down their choice. 

Note: You can respond with just "Pizza!" if you want to get straight to it, but this response is more helpful because it shows you're thinking about the user's preferences and wants to help them make a decision. If you just want to order pizza without considering the type, you can also say "Okay, I'm thinking of [type] pizza. Is that right?" 

Also, you can add some additional details like "With [ingredient]", "Without [ingredient]", or "With special sauce" to make the request more specific. F

In [90]:
# 6. Saving
huggingface_model_name = "varun2480/Marco-Llama-3.2-1B"
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")
model.push_to_hub(huggingface_model_name, token = os.getenv("HF_TOKEN"))
tokenizer.push_to_hub(huggingface_model_name, token = os.getenv("HF_TOKEN"))

Saved model to https://huggingface.co/varun2480/Marco-Llama-3.2-1B


In [92]:
# Merge to 4bit

# if True: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
# if True: model.push_to_hub_merged(huggingface_model_name, tokenizer, save_method = "merged_4bit", token = os.getenv("HF_TOKEN"))

if True: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit_forced",)
if True: model.push_to_hub_merged(huggingface_model_name, tokenizer, save_method = "merged_4bit_forced", token = os.getenv("HF_TOKEN"))


Unsloth: Merging 4bit and LoRA weights to 4bit...
This might take 5 minutes...


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:355: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


Done.
Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 10 minutes for Llama-7b... Done.
Unsloth: Merging 4bit and LoRA weights to 4bit...
This might take 5 minutes...
Done.
Unsloth: Saving 4bit Bitsandbytes model. Please wait...


No files have been modified since last commit. Skipping to prevent empty commit.


Saved merged_4bit model to https://huggingface.co/varun2480/Marco-Llama-3.2-1B


In [93]:
from unsloth import FastLanguageModel
import os
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "lora_model",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model)

model.push_to_hub_gguf(
    "varun2480/Marco-Llama-3.1-4B-bnb-4bit-python", # Change this to your username!
    tokenizer,
    quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
    token = os.getenv("HF_TOKEN"),
)

==((====))==  Unsloth 2025.1.8: Fast Llama patching. Transformers: 4.48.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 1.1G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 4.28 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 16/16 [00:01<00:00, 15.86it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving varun2480/Marco-Llama-3.1-4B-bnb-4bit-python/pytorch_model.bin...
Done.


...

==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m', 'q8_0', 'q5_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at varun2480/Marco-Llama-3.1-4B-bnb-4bit-python into f16 GGUF format.
The output location will be /content/varun2480/Marco-Llama-3.1-4B-bnb-4bit-python/unsloth.F16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: Marco-Llama-3.1-4B-bnb-4bit-python
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {32}
INFO:hf-to-gguf:gguf: loading mo

No files have been modified since last commit. Skipping to prevent empty commit.


Saved GGUF to https://huggingface.co/varun2480/Marco-Llama-3.1-4B-bnb-4bit-python
Unsloth: Uploading GGUF to Huggingface Hub...


No files have been modified since last commit. Skipping to prevent empty commit.


Saved GGUF to https://huggingface.co/varun2480/Marco-Llama-3.1-4B-bnb-4bit-python
